In [1]:
# load the new file
import pandas as pd

from datetime import date
import calendar

datasett = pd.read_csv('timeseries3.csv', sep=',', header=0, infer_datetime_format=True, parse_dates=True, index_col=0)
from datetime import date
datasett['Date'] = datasett['Date'].astype('datetime64[ns]')
datasett['month'] = datasett['Date'].apply(lambda x: x.month)

datasett['time_of_day'] = datasett['Date'].apply(lambda x: x.time())
datasett['hour_of_day'] = datasett['time_of_day'].apply(lambda x: x.hour)
datasett = pd.get_dummies(datasett, columns=['hour_of_day'])
datasett = datasett.drop(['time_of_day'], axis=1)

datasett['low_time'] = datasett['hour_of_day_2'] | datasett['hour_of_day_3'] | datasett['hour_of_day_4'] | datasett['hour_of_day_5']
datasett['high_time'] = datasett['hour_of_day_18'] | datasett['hour_of_day_19'] | datasett['hour_of_day_20']

datasett = datasett.drop(['month'], axis=1)

datasett = datasett.drop(['hour_of_day_0'], axis=1)
datasett = datasett.drop(['hour_of_day_1'], axis=1)
datasett = datasett.drop(['hour_of_day_2'], axis=1)
datasett = datasett.drop(['hour_of_day_3'], axis=1)
datasett = datasett.drop(['hour_of_day_4'], axis=1)
datasett = datasett.drop(['hour_of_day_5'], axis=1)
datasett = datasett.drop(['hour_of_day_6'], axis=1)
datasett = datasett.drop(['hour_of_day_7'], axis=1)
datasett = datasett.drop(['hour_of_day_8'], axis=1)
datasett = datasett.drop(['hour_of_day_9'], axis=1)
datasett = datasett.drop(['hour_of_day_10'], axis=1)
datasett = datasett.drop(['hour_of_day_11'], axis=1)
datasett = datasett.drop(['hour_of_day_12'], axis=1)
datasett = datasett.drop(['hour_of_day_13'], axis=1)
datasett = datasett.drop(['hour_of_day_14'], axis=1)
datasett = datasett.drop(['hour_of_day_15'], axis=1)
datasett = datasett.drop(['hour_of_day_16'], axis=1)
datasett = datasett.drop(['hour_of_day_17'], axis=1)
datasett = datasett.drop(['hour_of_day_18'], axis=1)
datasett = datasett.drop(['hour_of_day_19'], axis=1)
datasett = datasett.drop(['hour_of_day_20'], axis=1)
datasett = datasett.drop(['hour_of_day_21'], axis=1)
datasett = datasett.drop(['hour_of_day_22'], axis=1)
datasett = datasett.drop(['hour_of_day_23'], axis=1)

datasett = datasett.drop(['Date'], axis=1)
datasett.head(3)

,VAR,dewpti,hum,pressurei,rain,tempi,thunder,Day,Holiday,PCA_1,low_time,high_time
DateIdx,,,,,,,,,,,,
2016-05-01 00:00:00,24.25751,62.30,82.000000,29.960000,0.0,67.500000,0.0,1,0,-15.419709,0,0
2016-05-01 00:15:00,25.38191,49.85,84.458356,29.832557,0.0,67.257999,0.0,1,0,198.166945,0,0
2016-05-01 00:30:00,25.10542,62.60,88.000000,29.950000,0.0,66.200000,0.0,1,0,104.502454,0,0


In [2]:
number_of_days = 350
epochs_nbr     = 30
n_input        = 672
starting_point=34272-(number_of_days*96)

# normal test
i1, i2, i3, i4 = starting_point, 34272, 34272, 34368

# try day before the normal test
# i1, i2, i3, i4 = starting_point - 96, 34272 - 96, 34272 - 96, 34368 - 96

# try 2 days before the normal test
# i1, i2, i3, i4 = starting_point - 96*2, 34272 - 96*2, 34272 - 96*2, 34368 - 96*2

# try 3 days before the normal test
# i1, i2, i3, i4 = starting_point - 96*3, 34272 - 96*3, 34272 - 96*3, 34368 - 96*3


# multi headed multi-step cnn for the power usage dataset
from math import sqrt
from numpy import split
from numpy import array
from pandas import read_csv
from sklearn.metrics import mean_squared_error
from matplotlib import pyplot
from keras.utils.vis_utils import plot_model
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.models import Model
from keras.layers import Input
from keras.layers.merge import concatenate

# split a univariate dataset into train/test sets
def split_dataset(data):
    # split into standard weeks
    train, test = data[i1:i2], data[i3:i4]
    # restructure into windows of weekly data
    train = array(split(train, len(train)/96))
    test = array(split(test, len(test)/96))
    return train, test

# evaluate one or more weekly forecasts against expected values
def evaluate_forecasts(actual, predicted):
    scores = list()
    # calculate an RMSE score for each day
    for i in range(actual.shape[1]): # shape[1] gets the number rows
        # calculate mse
        mse = mean_squared_error(actual[:, i], predicted[:, i]) # get i'th column, and all rows
        # calculate rmse
        rmse = sqrt(mse)
        # store
        scores.append(rmse)
    # calculate overall RMSE
    s = 0
    for row in range(actual.shape[0]):
        for col in range(actual.shape[1]):
            s += (actual[row, col] - predicted[row, col])**2
    score = sqrt(s / (actual.shape[0] * actual.shape[1]))
    return score, scores

# summarize scores
def summarize_scores(name, score, scores):
    s_scores = ', '.join(['%.1f' % s for s in scores])
    print('%s: [%.3f] %s' % (name, score, s_scores))

# convert history into inputs and outputs
def to_supervised(train, n_input, n_out=96):
    # flatten data
    data = train.reshape((train.shape[0]*train.shape[1], train.shape[2]))
    X, y = list(), list()
    in_start = 0
    # step over the entire history one time step at a time
    for _ in range(len(data)):
        # define the end of the input sequence
        in_end = in_start + n_input
        out_end = in_end + n_out
        # ensure we have enough data for this instance
        if out_end <= len(data):
            X.append(data[in_start:in_end, :])
            y.append(data[in_end:out_end, 0])
        # move along one time step
        in_start += 1
    return array(X), array(y)

# train the model
def build_model(train, n_input):
    # prepare data
    train_x, train_y = to_supervised(train, n_input)
    # define parameters
    verbose, epochs, batch_size = 2, epochs_nbr, 16
    n_timesteps, n_features = train_x.shape[1], train_x.shape[2]
    n_outputs = train_y.shape[1]
    # create a channel for each variable
    in_layers, out_layers = list(), list()
    for _ in range(n_features):
        inputs = Input(shape=(n_timesteps,1))
        conv1 = Conv1D(32, 3, activation='relu')(inputs)
        conv2 = Conv1D(32, 3, activation='relu')(conv1)
        pool1 = MaxPooling1D()(conv2)
        flat = Flatten()(pool1)
        # store layers
        in_layers.append(inputs)
        out_layers.append(flat)
    # merge heads
    merged = concatenate(out_layers)
    # interpretation
    dense1 = Dense(200, activation='relu')(merged)
    dense2 = Dense(100, activation='relu')(dense1)
    outputs = Dense(n_outputs)(dense2)
    model = Model(inputs=in_layers, outputs=outputs)
    # compile model
    model.compile(loss='mse', optimizer='adam')
    # fit network
    input_data = [train_x[:,:,i].reshape((train_x.shape[0],n_timesteps,1)) 
                  for i in range(n_features)]
    model.fit(input_data, train_y, epochs=epochs, 
              batch_size=batch_size, verbose=verbose)
    return model

# make a forecast
def forecast(model, history, n_input):
    # flatten data
    data = array(history)
    data = data.reshape((data.shape[0]*data.shape[1], data.shape[2]))
    # retrieve last observations for input data
    input_x = data[-n_input:, :]
    # reshape into n input arrays
    input_x = [input_x[:,i].reshape((1,input_x.shape[0],1)) for i in range(input_x.shape[1])]
    # forecast the next week
    yhat = model.predict(input_x, verbose=2)
    # we only want the vector forecast
    yhat = yhat[0]
    return yhat

# load the new file
dataset = datasett

# split into train and test
train, test = split_dataset(dataset.values)

# evaluate model and get scores

#score, scores = evaluate_model(train, test, n_input)


Using TensorFlow backend.


In [4]:
yhat_sequence.tolist()

[21.43974494934082,
 21.35433006286621,
 21.456422805786133,
 21.563411712646484,
 21.629920959472656,
 21.54561424255371,
 21.442773818969727,
 21.336212158203125,
 21.116044998168945,
 21.19214630126953,
 20.790861129760742,
 20.529294967651367,
 20.236862182617188,
 20.14304542541504,
 19.826204299926758,
 19.53729820251465,
 19.453079223632812,
 19.321828842163086,
 19.268226623535156,
 18.962446212768555,
 18.528297424316406,
 18.13018798828125,
 17.701148986816406,
 17.352025985717773,
 16.9786434173584,
 16.82727813720703,
 16.680095672607422,
 16.641740798950195,
 16.79139518737793,
 17.067047119140625,
 17.37692642211914,
 17.87432861328125,
 18.669706344604492,
 19.07497787475586,
 19.7342472076416,
 20.310012817382812,
 20.91786003112793,
 21.319677352905273,
 21.504121780395508,
 21.661882400512695,
 21.794178009033203,
 21.8226261138916,
 21.85040283203125,
 21.80544090270996,
 21.821189880371094,
 21.74934959411621,
 21.590518951416016,
 21.33043670654297,
 21.32560157775

In [6]:
next_week = forecast(model, history, n_input)
next_week.tolist()

[21.170400619506836,
 21.084739685058594,
 21.126239776611328,
 21.20359230041504,
 21.238359451293945,
 21.065258026123047,
 20.837303161621094,
 20.62456703186035,
 20.322912216186523,
 20.350330352783203,
 19.89147186279297,
 19.54164695739746,
 19.202707290649414,
 18.952856063842773,
 18.528785705566406,
 18.183460235595703,
 18.09791374206543,
 18.074634552001953,
 18.206335067749023,
 18.156829833984375,
 18.143293380737305,
 18.28551483154297,
 18.419567108154297,
 18.796079635620117,
 19.1159610748291,
 19.699132919311523,
 20.177223205566406,
 20.697202682495117,
 21.237133026123047,
 21.759349822998047,
 22.04967498779297,
 22.337528228759766,
 22.81922721862793,
 22.694957733154297,
 22.71135711669922,
 22.60078239440918,
 22.488000869750977,
 22.22136688232422,
 21.754140853881836,
 21.440248489379883,
 21.17209243774414,
 20.956661224365234,
 20.933069229125977,
 20.97305679321289,
 21.167640686035156,
 21.348358154296875,
 21.533899307250977,
 21.59904670715332,
 21.9635

Double check the length of the training set before and after appending the test set.

In [6]:
history = [x for x in train]
predictions = list()
len(history)

252

In [7]:
for i in range(len(test)):
    # predict the week
    yhat_sequence = forecast(model, history, n_input)
    # store the predictions
    predictions.append(yhat_sequence)
    # get real observation and add to history for predicting the next week
    history.append(test[i, :])
len(history)

253